In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

print("🚀 Environment is ready!")


In [ ]:
import pandas as pd

# Full path to your dataset
file_path = r"C:\Users\yuvra\OneDrive\Desktop\marketing_ab_test\data\A_B_testing_dataset.csv"

# Load dataset
df = pd.read_csv(file_path)

# Preview first 5 rows
print(df.head())

# Check unique campaign values
print(df['campaign'].unique())



In [ ]:
print(df.columns)


In [ ]:
print(df.head(10))


In [ ]:
import pandas as pd

# Create Facebook subset
facebook = df[[
    "date_of_campaign",
    "facebook_ad_views",
    "facebook_ad_clicks",
    "facebook_ad_conversions",
    "facebook_cost_per_ad",
    "facebook_ctr",
    "facebook_conversion_rate",
    "facebook_cost_per_click"
]].copy()

facebook["campaign"] = "Facebook"
facebook = facebook.rename(columns={
    "facebook_ad_views": "views",
    "facebook_ad_clicks": "clicks",
    "facebook_ad_conversions": "conversions",
    "facebook_cost_per_ad": "cost_per_ad",
    "facebook_ctr": "ctr",
    "facebook_conversion_rate": "conversion_rate",
    "facebook_cost_per_click": "cpc"
})

# Create AdWords subset
adwords = df[[
    "date_of_campaign",
    "adword_ad_views",
    "adword_ad_clicks",
    "adword_ad_conversions",
    "adword_cost_per_ad",
    "adword_ctr",
    "adword_conversion_rate",
    "adword_cost_per_click"
]].copy()

adwords["campaign"] = "AdWords"
adwords = adwords.rename(columns={
    "adword_ad_views": "views",
    "adword_ad_clicks": "clicks",
    "adword_ad_conversions": "conversions",
    "adword_cost_per_ad": "cost_per_ad",
    "adword_ctr": "ctr",
    "adword_conversion_rate": "conversion_rate",
    "adword_cost_per_click": "cpc"
})

# Combine into one long dataframe
df_long = pd.concat([facebook, adwords], ignore_index=True)

print(df_long.head(10))
print(df_long["campaign"].unique())




In [ ]:
summary = df_long.groupby("campaign").agg(
    total_clicks=("clicks", "sum"),
    total_conversions=("conversions", "sum")
).reset_index()

# Calculate conversion rate
summary["conversion_rate"] = summary["total_conversions"] / summary["total_clicks"]

print(summary)


In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# Successes = conversions
successes = summary["total_conversions"].values

# Observations = clicks
nobs = summary["total_clicks"].values

z_stat, p_value = proportions_ztest(successes, nobs)

print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_value:.4f}")


In [ ]:
# Conversion rates
fb_rate = summary.loc[summary["campaign"] == "Facebook", "conversion_rate"].values[0]
ad_rate = summary.loc[summary["campaign"] == "AdWords", "conversion_rate"].values[0]

lift = ((fb_rate - ad_rate) / ad_rate) * 100

print(f"Facebook Conversion Rate: {fb_rate:.2%}")
print(f"AdWords Conversion Rate: {ad_rate:.2%}")
print(f"Lift: {lift:.2f}%")


In [ ]:
# Add spend per campaign
spend_summary = df_long.groupby("campaign").agg(
    total_cost=("cost_per_ad", "sum"),
    total_conversions=("conversions", "sum")
).reset_index()

# Assume each conversion = $100 revenue (you can change this assumption)
revenue_per_conversion = 100
spend_summary["revenue"] = spend_summary["total_conversions"] * revenue_per_conversion
spend_summary["roi"] = (spend_summary["revenue"] - spend_summary["total_cost"]) / spend_summary["total_cost"]

print(spend_summary[["campaign", "total_cost", "revenue", "roi"]])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Conversion rates (from your summary earlier)
conv_rates = summary[["campaign", "conversion_rate"]]

plt.figure(figsize=(6,4))
sns.barplot(data=conv_rates, x="campaign", y="conversion_rate", palette="viridis")
plt.title("Conversion Rate Comparison", fontsize=14)
plt.ylabel("Conversion Rate (%)")
plt.xlabel("Campaign")
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
plt.show()


In [ ]:
roi_data = spend_summary[["campaign", "roi"]]

plt.figure(figsize=(6,4))
sns.barplot(data=roi_data, x="campaign", y="roi", palette="magma")
plt.title("ROI Comparison", fontsize=14)
plt.ylabel("ROI (%)")
plt.xlabel("Campaign")
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
plt.show()


In [ ]:
facebook_cvr = summary.loc[summary["campaign"]=="Facebook", "total_conversions"].values[0] / summary.loc[summary["campaign"]=="Facebook", "total_clicks"].values[0]
adwords_cvr = summary.loc[summary["campaign"]=="AdWords", "total_conversions"].values[0] / summary.loc[summary["campaign"]=="AdWords", "total_clicks"].values[0]

lift = (facebook_cvr / adwords_cvr - 1) * 100



In [ ]:
from statsmodels.stats.proportion import proportions_ztest

successes = summary["total_conversions"].values
nobs = summary["total_clicks"].values

z_stat, p_value = proportions_ztest(successes, nobs)


In [ ]:
import json

results = {
    "facebook_cvr": round(facebook_cvr * 100, 2),
    "adwords_cvr": round(adwords_cvr * 100, 2),
    "lift": round(lift, 2),
    "p_value": round(p_value, 4)
}

with open("results.json", "w") as f:
    json.dump(results, f)

print("✅ Results exported to results.json")


In [ ]:
import json
import os

# Save directly into the dashboard folder
output_path = r"C:\Users\yuvra\OneDrive\Desktop\marketing_ab_test\dashboard\results.json"

results = {
    "facebook_cvr": round(facebook_cvr * 100, 2),
    "adwords_cvr": round(adwords_cvr * 100, 2),
    "lift": round(lift, 2),
    "p_value": round(p_value, 4)
}

with open(output_path, "w") as f:
    json.dump(results, f)

print(f"✅ Results exported to {output_path}")


In [ ]:
# Ensure we have spend column in df_long
if "cost_per_ad" in df_long.columns and "clicks" in df_long.columns:
    df_long["spend"] = df_long["clicks"] * df_long["cost_per_ad"]
elif "cpc" in df_long.columns and "clicks" in df_long.columns:
    df_long["spend"] = df_long["clicks"] * df_long["cpc"]
else:
    raise ValueError("Could not find cost columns to calculate spend")


In [ ]:
summary = df_long.groupby("campaign").agg(
    total_views=("views","sum"),
    total_clicks=("clicks","sum"),
    total_conversions=("conversions","sum"),
    total_spend=("spend","sum")
).reset_index()


In [ ]:
# Add metrics
summary["conversion_rate"] = summary["total_conversions"] / summary["total_clicks"]
summary["ctr"] = summary["total_clicks"] / summary["total_views"]

# Assume each conversion gives revenue of 10 (you can change this later)
revenue_per_conversion = 10
summary["roi"] = (summary["total_conversions"] * revenue_per_conversion) - summary["total_spend"]


In [ ]:
import json

summary_json = {
    "facebook_cvr": float(round(summary.loc[summary["campaign"]=="Facebook", "conversion_rate"].values[0]*100, 2)),
    "adwords_cvr": float(round(summary.loc[summary["campaign"]=="AdWords", "conversion_rate"].values[0]*100, 2)),
    "lift": float(round(((summary.loc[summary["campaign"]=="Facebook", "conversion_rate"].values[0] -
                          summary.loc[summary["campaign"]=="AdWords", "conversion_rate"].values[0]) /
                          summary.loc[summary["campaign"]=="AdWords", "conversion_rate"].values[0]) * 100, 2)),
    "facebook_roi": float(round(summary.loc[summary["campaign"]=="Facebook", "roi"].values[0], 2)),
    "adwords_roi": float(round(summary.loc[summary["campaign"]=="AdWords", "roi"].values[0], 2))
}

with open("../dashboard/results.json", "w") as f:
    json.dump(summary_json, f)


In [ ]:
timeseries = {
    "dates": df_long["date_of_campaign"].unique().tolist(),
    "facebook": {
        "views": df_long[df_long["campaign"]=="Facebook"]["views"].tolist(),
        "clicks": df_long[df_long["campaign"]=="Facebook"]["clicks"].tolist(),
        "conversions": df_long[df_long["campaign"]=="Facebook"]["conversions"].tolist()
    },
    "adwords": {
        "views": df_long[df_long["campaign"]=="AdWords"]["views"].tolist(),
        "clicks": df_long[df_long["campaign"]=="AdWords"]["clicks"].tolist(),
        "conversions": df_long[df_long["campaign"]=="AdWords"]["conversions"].tolist()
    }
}

import json
with open("../dashboard/timeseries.json", "w") as f:
    json.dump(timeseries, f)


In [ ]:
# --- Confidence Intervals for each campaign's conversion rate ---
from statsmodels.stats.proportion import proportion_confint, confint_proportions_2indep
import numpy as np

# Raw counts
fb_conv  = int(summary.loc[summary["campaign"]=="Facebook","total_conversions"].values[0])
fb_click = int(summary.loc[summary["campaign"]=="Facebook","total_clicks"].values[0])
aw_conv  = int(summary.loc[summary["campaign"]=="AdWords","total_conversions"].values[0])
aw_click = int(summary.loc[summary["campaign"]=="AdWords","total_clicks"].values[0])

# Wilson CIs for proportions (more stable than normal approx)
fb_cil, fb_ciu = proportion_confint(fb_conv, fb_click, method="wilson")
aw_cil, aw_ciu = proportion_confint(aw_conv, aw_click, method="wilson")

# 95% CI for difference in proportions (Facebook - AdWords)
diff_ci_low, diff_ci_high = confint_proportions_2indep(fb_conv, fb_click, aw_conv, aw_click, method="wald")

# Hedge’s g is not appropriate for proportions; use Cohen’s h (effect size for proportions)
# Cohen’s h = 2*arcsin(sqrt(p1)) - 2*arcsin(sqrt(p2))
p1 = fb_conv / fb_click
p2 = aw_conv / aw_click
cohens_h = 2*np.arcsin(np.sqrt(p1)) - 2*np.arcsin(np.sqrt(p2))

ci_payload = {
    "facebook_cvr_ci": [round(fb_cil*100,2), round(fb_ciu*100,2)],
    "adwords_cvr_ci":  [round(aw_cil*100,2), round(aw_ciu*100,2)],
    "diff_cvr_ci":     [round((diff_ci_low)*100,2), round((diff_ci_high)*100,2)],
    "cohens_h":        float(cohens_h)
}
ci_payload


In [ ]:
from statsmodels.stats.power import NormalIndPower

power_calc = NormalIndPower()

# Compute required sample size per group to detect the observed effect size
required_n_per_group = power_calc.solve_power(
    effect_size=abs(cohens_h),
    nobs1=None,        # this is what we want to solve for
    alpha=0.05,
    power=0.80,
    ratio=1.0,
    alternative='two-sided'
)

required_n_per_group = int(np.ceil(required_n_per_group))
print("Required clicks per group (80% power):", required_n_per_group)


In [ ]:
summary_json = {
    "facebook_cvr": float(round(p1*100, 2)),
    "adwords_cvr": float(round(p2*100, 2)),
    "lift": float(round(((p1 - p2)/p2)*100, 2)),
    "p_value": float(round(p_value, 6)),
    "facebook_roi": float(round(summary.loc[summary["campaign"]=="Facebook","roi"].values[0], 2)),
    "adwords_roi": float(round(summary.loc[summary["campaign"]=="AdWords","roi"].values[0], 2)),
    "facebook_cvr_ci": ci_payload["facebook_cvr_ci"],
    "adwords_cvr_ci": ci_payload["adwords_cvr_ci"],
    "diff_cvr_ci": ci_payload["diff_cvr_ci"],
    "cohens_h": ci_payload["cohens_h"],
    "required_clicks_per_group_80p": int(required_n_per_group)
}

import json
with open("../dashboard/results.json", "w") as f:
    json.dump(summary_json, f)


In [ ]:
df_long["date_of_campaign"] = pd.to_datetime(df_long["date_of_campaign"])
df_long["weekday"] = df_long["date_of_campaign"].dt.day_name()
df_long["year_month"] = df_long["date_of_campaign"].dt.to_period("M").astype(str)

seg_weekday = (
    df_long.groupby(["campaign","weekday"])
    .agg(views=("views","sum"), clicks=("clicks","sum"), conversions=("conversions","sum"), spend=("spend","sum"))
    .reset_index()
)
seg_weekday["cvr"] = seg_weekday["conversions"] / seg_weekday["clicks"]
seg_weekday["ctr"] = seg_weekday["clicks"] / seg_weekday["views"]

seg_month = (
    df_long.groupby(["campaign","year_month"])
    .agg(views=("views","sum"), clicks=("clicks","sum"), conversions=("conversions","sum"), spend=("spend","sum"))
    .reset_index()
)
seg_month["cvr"] = seg_month["conversions"] / seg_month["clicks"]
seg_month["ctr"] = seg_month["clicks"] / seg_month["views"]

# Export segmentation payload (keep small & dashboard-friendly)
seg_payload = {
    "weekday": seg_weekday.to_dict(orient="records"),
    "month": seg_month.to_dict(orient="records")
}
with open("../dashboard/segments.json", "w") as f:
    json.dump(seg_payload, f)


In [ ]:
alerts = []

alpha = 0.05
sig = (p_value < alpha)
practical = (summary_json["lift"] > 10)  # tune this threshold

if sig and practical:
    alerts.append({
        "level": "success",
        "title": "Facebook significantly outperforms AdWords",
        "detail": f"Lift = {summary_json['lift']}%, p = {summary_json['p_value']}"
    })
elif sig and not practical:
    alerts.append({
        "level": "warning",
        "title": "Statistically significant but small lift",
        "detail": f"Lift = {summary_json['lift']}%, p = {summary_json['p_value']}"
    })
else:
    alerts.append({
        "level": "info",
        "title": "No significant difference yet",
        "detail": f"Current p = {summary_json['p_value']} (α = 0.05)"
    })

# Early stopping heuristic (very simple Bonferroni at 3 checkpoints)
# If you're checking 3 times, use alpha/3 at each interim check
interim_alpha = alpha / 3
if p_value < interim_alpha:
    alerts.append({
        "level": "success",
        "title": "Early stopping recommendation",
        "detail": f"Strong significance (p < {interim_alpha:.3f}). Consider stopping early."
    })

with open("../dashboard/alerts.json", "w") as f:
    json.dump(alerts, f)
